In [139]:
import pandas as pd
import numpy as np
import requests
import s3fs
import MidParentValue as MP
import rulefilter as rf
import utils
import master_table as mt
import collections
import re
import os
import warnings
warnings.filterwarnings("ignore")
import dask
from dask import delayed
import dask.dataframe as dd
from functools import reduce
import itertools
import time
import json
import hvac
import boto3

In [140]:
if 'ec2' in os.environ['HOME']:
    master_file = utils.read_as_dask_1('ycao1/AoB/SweetCorn/Tropical/ABLUP/ABLUP_GCA_df_oneyear.csv', bucket='veg-apd-sdi-predictiveanalytcs-prod-workspace')
else:
    master_file = utils.read_as_dask('ycao1/AoB/SweetCorn/Tropical/ABLUP/ABLUP_GCA_df_oneyear.csv', bucket='veg-apd-sdi-predictiveanalytcs-prod-workspace')

In [141]:
master_file = master_file.compute().set_index(pd.Index(range(master_file.compute().shape[0])))

In [142]:
# Generate the master table
import master_table as mt

id_table = utils.read_s3_file_generic(filename = 'Corn/Corn_IDs.csv', bucket = 'veg-apd-sdi-predictiveanalytcs-prod-reference-data')
master_file_table = mt.combine_blup_id(master_file, id_table)

marker_data = utils.read_s3_file_generic(filename = 'Corn/FTS_MarkerData_SweetCorn_2019.csv', bucket = 'veg-apd-sdi-predictiveanalytcs-prod-reference-data')

master_file_table = mt.combine_blup_markerData(master_file_table, marker_data)

parentalTable = utils.read_s3_file_generic(filename = 'Corn/Corn_Parentals.csv', bucket = 'veg-apd-sdi-predictiveanalytcs-prod-reference-data')

master_file_table = mt.combine_blup_parental(master_file_table, parentalTable)

stageData = utils.read_s3_file_generic(filename = 'ycao1/AoB/SweetCorn/Tropical/stage_count_dat.csv', bucket = 'veg-apd-sdi-predictiveanalytcs-prod-workspace')
master_file_table = mt.combine_blup_stage(master_file_table, stageData)

gpc_df = utils.read_s3_file_generic(filename = 'GPC_Data/Corn/Corn_GPC.csv', bucket = 'veg-apd-sdi-predictiveanalytcs-prod-pheno-data')
master_file_table = mt.combine_blup_obs(master_file_table, gpc_df)

master_file_table.to_csv('/mnt/TropicalSC/master_file_table_oneyear.csv')

utils.upload_file(file_name = "/mnt/TropicalSC/master_file_table_oneyear.csv", bucket="veg-apd-sdi-predictiveanalytcs-prod-workspace",
            object_name = 'ycao1/AoB/SweetCorn/master_table/TropicalSC_ABLUP_GCA_master_table_oneyear.csv')

_______Combine Id_Table_______
Initial Size        (5670, 170) (2100493, 13)
Removed 0 null germplasm
Master Shape:       (6118, 177)
_____Combine Marker_Table_____
Initial Size        (6118, 177) (26187, 8)
Master Shape:       (6118, 177)
____Combine Parental_Table____
Initial Size        (6118, 177) (1463898, 11)
Master Shape:       (6118, 179)
_____Combine Stage_Table______
Initial Size        (6118, 179) (12209, 3)
Master Shape:       (6118, 185)
______Combine GPC_Table_______
Initial Size        (6118, 185) (644842, 111)
Master Shape:       (6118, 249)


True

In [143]:
master_file_table = utils.read_as_dask('ycao1/AoB/SweetCorn/master_table/TropicalSC_ABLUP_GCA_master_table_oneyear.csv', 
                                    bucket = 'veg-apd-sdi-predictiveanalytcs-prod-workspace')
        
        # add index
master_file_table =  master_file_table.compute().set_index(pd.Index(range(master_file_table.compute().shape[0])))

In [144]:
key_inbred = pd.read_csv('/mnt/TropicalSC/inbredline.csv')
master_file_filtered = master_file_table[master_file_table['PEDIGREE_NAME'].isin(key_inbred['Pedigree'])]

In [145]:
master_file_filtered = rf.filter_by_HETGP(master_file_filtered, het=['A', 'B', 'AB'])

In [146]:
########### Getting Trait list 
# master_file_filtered = master_file
# traits = [col for col in master_file_filtered.columns if '_predicted.value' in col]
traits = [col for col in master_file_filtered.columns if 'predicted.value_' in col]
traits_1 = []
for tt in traits: 
    t_1 = re.sub('predicted.value_', '', tt)
#     if (t_1 != 'SC_RWCT') & (t_1 != 'FBRIX'):
    traits_1.append(t_1)

In [147]:
test = MP.gather_mid_value(dd.from_pandas(master_file_filtered, npartitions=3), traits_1)


In [148]:
reference_cols = [col for col in master_file_filtered.columns if not any(tr in col for tr in traits_1)]
reference_table = master_file_filtered[reference_cols]
reference_table = reference_table.drop(columns=['PEDIGREE'], axis=1)
reference_table = reference_table.drop_duplicates(subset='PEDIGREE_NAME', keep='last') # there are duplicates

In [149]:
diallel_table = utils.add_reference_dat(test, reference_table)

In [150]:
diallel_table['clean_origin'] = diallel_table['P1_M.GERMPLASM.ORIGIN'] + '__' + diallel_table['P2_M.GERMPLASM.ORIGIN']
diallel_table['clean_pedigree'] = diallel_table['P1_PEDIGREE'] + '__' + diallel_table['P2_PEDIGREE']

In [151]:
with open('/mnt/TropicalSC/config.txt') as f:
        DEFAULTS = json.loads(f.read())

In [152]:
DEFAULTS

{'Filters': {'diallel': {'rules': [{'trait': 'HETGP_GCV',
     'operator': 'in',
     'value': 'A_A,B_B,AB_AB,A_AB,B_AB,AB_A,AB_B'},
    {'trait': 'SC_INBRDSTG_GCV',
     'operator': 'in',
     'value': 'PS1_SCR1,SCR1_PS1,PS1_SCR2,SCR2_PS1,PS1_PS1,PS2_SCR1,SCR1_PS2,PS2_SCR2,SCR2_PS2,PS2_PS1,PS1_PS2,PS2_PS2,PS2_PS3,PS3_PS2,PS2_PS4,PS4_PS2,PS2_CM,CM_PS2,PS3_SCR1,SCR1_PS3,PS3_SCR2,SCR2_PS3,PS3_PS1,PS1_PS3,PS3_PS3,PS3_PS4,PS4_PS3,PS3_CM,CM_PS3,PS4_SCR1,SCR1_PS4,PS4_SCR2,SCR2_PS4,PS4_PS1,PS1_PS4,PS4_PS4'}]}}}

In [153]:
diallel_table = utils.create_column_combo(diallel_table, DEFAULTS)
DataObject = collections.namedtuple('DataObject', 'data')
in_source = {}
in_source['diallel'] = DataObject(data = diallel_table)
in_source_filtered = rf.apply_rules(in_source, DEFAULTS['Filters'])
####################### Write results to a csv file
print('\n========= Writing Results to a Local folder =========')
# in_source_filtered['diallel'].data.to_csv('/mnt/TropicalSC/diallel_table_filtered_threeyear.csv')

HETGP_GCV
SC_INBRDSTG_GCV
diallel table: 
		shape: (1378, 227)
-----------------------------------
HETGP_GCV in A_A,B_B,AB_AB,A_AB,B_AB,AB_A,AB_B 
		shape: (748, 227)
SC_INBRDSTG_GCV in PS1_SCR1,SCR1_PS1,PS1_SCR2,SCR2_PS1,PS1_PS1,PS2_SCR1,SCR1_PS2,PS2_SCR2,SCR2_PS2,PS2_PS1,PS1_PS2,PS2_PS2,PS2_PS3,PS3_PS2,PS2_PS4,PS4_PS2,PS2_CM,CM_PS2,PS3_SCR1,SCR1_PS3,PS3_SCR2,SCR2_PS3,PS3_PS1,PS1_PS3,PS3_PS3,PS3_PS4,PS4_PS3,PS3_CM,CM_PS3,PS4_SCR1,SCR1_PS4,PS4_SCR2,SCR2_PS4,PS4_PS1,PS1_PS4,PS4_PS4 
		shape: (619, 227)

========= Writing Results to a Local folder =========


In [154]:
# diallel_table.to_csv('/mnt/TropicalSC/dialle_table_full_three.csv')

In [155]:
########## Check if an origin or a hybrid has been create in the past ############
parentalTable['Pedigree_1'] = parentalTable['FemalePedigree'] + '__' + parentalTable['MalePedigree']
parentalTable['Pedigree_2'] = parentalTable['MalePedigree'] + '__' + parentalTable['FemalePedigree']

In [156]:
diallel_table['existing_status'] = [True if (x in parentalTable['Pedigree_1']) or (x in parentalTable['Pedigree_2']) else False for x in diallel_table['clean_pedigree']]

In [157]:
diallel_table = rf.remove_same_origin(diallel_table)

In [159]:
diallel_table.to_csv('/mnt/TropicalSC/dialle_table_full_oneyear_08122021.csv')

In [160]:
in_source_filtered['diallel'].data.to_csv('/mnt/TropicalSC/diallel_table_filtered_oneyear_08122021.csv')